## NPT-SALIVA TESTING, COVID 19 
### Affan Shoukat 
#### Center for Disease Modelling and Analysis

In [1]:
# activate the folder environment to pull in the correct versions of the packages used
using Pkg
Pkg.activate(".")
Pkg.status()

 Activating environment at `/mnt/c/Users/affan/OneDrive/Documents/postdoc projects/2019-ncov/projects/npt-saliva-testing/Project.toml`


Status `/mnt/c/Users/affan/OneDrive/Documents/postdoc projects/2019-ncov/projects/npt-saliva-testing/Project.toml`
  [336ed68f] CSV v0.7.7
  [a93c6f00] DataFrames v0.21.8
  [31c24e10] Distributions v0.23.12
  [60bf3e95] GLPK v0.14.2
  [dc211083] Gnuplot v1.3.0
  [2fda8390] LsqFit v0.11.0
  [6fe1bfb0] OffsetArrays v1.3.1
  [429524aa] Optim v1.2.0
  [d96e819e] Parameters v0.12.1
  [8bb1440f] DelimitedFiles 
  [9a3f8284] Random 


In [10]:
using Revise
includet("model_functions.jl"); # if this part errors out, have to run Pkg.instantiate() to download the packages. See Julia's documentation on Environments.

# METHODOLOGY

In [2]:
## Plot the relative infectivity curves from Ashcroft and He papers 
## The order of these plots may be different in the main manuscript.

ymeans_ashcroft = get_ashcroft_curve(TVAL_OFFSET)
ymeans_nature = get_nature_curve(TVAL_OFFSET)

@gp "set term svg enhanced standalone mouse size 800,300" ## use jupyter
#@gp "set term png enhanced size 600,400" ## use jupyter
#@gp "set size 0.5,0.5" ## sets the size wrt to the canvas
@gp :- "set style fill transparent solid 0.2 noborder"
@gp :- tit="Relative Infectivity Curve"
@gp :- "set xlabel 'days from symptom onset"
@gp :- "set ylabel 'relative infectivity'"
@gp :- TVAL_OFFSET ymeans_ashcroft[2][TVAL_OFFSET] ymeans_ashcroft[3][TVAL_OFFSET] "with filledcu title 'range' lw 2 lc rgb 'red' "
@gp :- TVAL_OFFSET ymeans_ashcroft[1][TVAL_OFFSET] "w l title 'ashcroft' lw 3 lc rgb 'red'"
@gp :- TVAL_OFFSET ymeans_nature[TVAL_OFFSET] "w l title 'nature' lw 3 lc rgb 'blue'"

In [3]:
# plot sensitivity data 
# first column is digitized from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7151472/
# second column is digitized from https://faseb.onlinelibrary.wiley.com/doi/10.1096/fj.202001700RR
# for the plotting we only use the harvard data. 

col1 = OffsetArray(Int64.(sens_csv.study[2:26])./100, 1:25)
col2 = OffsetArray(Int64.(sens_csv.harvard[1:26])./100, 0:25)



@gp "set term svg enhanced standalone mouse size 600,400" ## use jupyter
@gp :- "set xlabel 'days post symptom onset'"
@gp :- "set ylabel 'percent positivity'"
@gp :- tit="Sensitivity/Percent Positivity" key="opaque" 
@gp :- 1:25 col1[1:25] "with points title 'Zhang et al' pt 7 pointsize 0.65 lc rgb 'blue'"
@gp :- 1:25 col1[1:25] "with lines notitle  lc rgb 'blue'"
@gp :- 0:25 col2[0:25] "with points title 'Lennerz et al' pt 7 pointsize 0.65 lc rgb 'red'"
@gp :- 0:25 col2[0:25] "with lines notitle  lc rgb 'red'"
display(@gp)

# check for optimizations
#@code_warntype get_sensitivity_data()
#@time get_sensitivity_data()

In [4]:
# The Gompertz function g(t) 
gmp_array = [gompertz(i, TVAL_OFFSET) for i = 1:10]
@gp "set term svg enhanced standalone mouse size 600,400" ## use jupyter
@gp :- "set xlabel 'symptom onset'"
for i in 1:length(gmp_array)
    @gp :- TVAL_OFFSET gmp_array[i][TVAL_OFFSET] "with lines title 'shift $i'"
end
@gp

In [5]:
## sampling of incubation periods - histogram
histdata = hist([infect_start() for _ = 1:1000], bs=1)
@gp "set term svg enhanced standalone mouse size 600,400" ## use jupyter
@gp :- "set style histogram cluster gap 2"
@gp :- "set style fill solid 0.1 border -1"
@gp :- "set xlabel 'incubation days'"
@gp :- "set ylabel 'counts'"
@gp :- "set boxwidth 0.9"
#@gp :- histdata.bins histdata.counts "w fillsteps notitle lc rgb 'red'"
@gp :- histdata.bins histdata.counts "w boxes smooth freq notitle lc rgb 'black'"

#@time infect_start()
#@code_warntype infect_start()

### Model and Fitting Process
We want to optimize the parameters of a modified hill function that maps the infectivity curve to the sensitivity of the test. In particular, 
$$
\min_{n, C} \sum_{t = 1}^{30} \left( s(t) - \left[ \frac{r(t)^n}{r(t)^n + C} \right] g_p(t) \right)^2
$$
where $s(t)$ is the sensitivity (digitized from [source](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7151472/)) and $g_p(t)$ is a shifted Gompertz function by $p$ days.

In [6]:
# the function fitted_curves() returns 13 sensitivity curves, one for each value of the incubation period from 2 to 13. 
fc = fitted_curves() # this gives us the sensitivity curves for NP tests
@gp "set term svg enhanced standalone mouse size 800,300" ## use jupyter
@gp :- "set xlabel 'symptom onset'"
@gp :- "set ylabel 'sensitivity"
for i in sort!(collect(keys(fc)))
    yvals = fc[i]
    @gp :- -15:40 yvals[-15:40] "with lines title 'inc period $i'"
end
@gp


In [7]:
# merge the raw sensivitiy data, the fitted sensitivity curve (for only 5 day incubation period, and the corresponding saliva test)

myhill = fc[5]# plot sensitivity curve with infectiouness starting at i days
println("maximum: $(maximum(myhill))")
get_saliva_curve(myhill) = rand(Uniform(0.70, 0.97)) * myhill / maximum(myhill)
    
saliva_curves = hcat([get_saliva_curve(myhill) for _ = 1:1000]...)
saliva_mean = dropdims(mean(saliva_curves, dims=2), dims=2)
slows = quantile.(eachrow(saliva_curves), [0.025])
shighs = quantile.(eachrow(saliva_curves), [0.975])

@gp "set term svg enhanced standalone mouse size 800,400" ## use jupyter
@gp :- tit="Fitted Hill Curve" key="opaque" 
#@gp :- TVAL_OFFSET ymeans_offset[TVAL_OFFSET] "with lines title 'rel infectivity'"
@gp :- TVAL_OFFSET slows shighs "with filledcu title 'sal range' lw 2 lc rgb '#a1d99b' "
@gp :- TVAL_OFFSET saliva_mean "with lines title 'saliva (5 days inc)' lc rgb 'black' lw 3"
@gp :- TVAL_OFFSET myhill[TVAL_OFFSET] "with boxes title 'npt (5 days inc)' lc rgb 'blue'"
@gp :- 0:25 get_sensitivity_data()[0:25] "with points title 'sensitivity' pt 7 pointsize 0.75 lc rgb 'red'" 

## make a dataframe for Seyed's plotting
#dd = DataFrame(time=TVAL_OFFSET, hill=myhill[TVAL_OFFSET], saliva_mean=saliva_mean, saliva_low=slows, saliva_high=shighs, raw_sensitivity=0.)
#dd.raw_sensitivity[16:41] = get_sensitivity_data()[0:25]
#CSV.write("optimized_hillfunction.csv", dd)



maximum: 0.9369092324402044


# RESULTS
 - results may be slightly different from published figure due to stochasticity
 - this jupyter cell only provides one of the scenarios. 
 - to run more scenarios, change the model params `mp` (see cell documentation) 
 - or alternatively, run the function `run_sims` in the code file
 - Manuscript figures were generated in Matlab

In [8]:
# Setup scenario parameters 
# struct ModelParams
#   test_type = :np or :sal (for nasal or saliva)
#   inf_type = :all or :pre (for entire infect or just presymptomatic stage)
#   firsttest::Int16 = 50   (date of first test)
#   testfreq::Int16 = 7     (frequency of testing)
#   exp_days::Int32 = 100   (total system days)
# or alternatively run run_sims() to run a whole panel of different scenarios. 
# note, this takes up considerable dataspace (a few hundred megabytes) 
# and be mindful of the save location of the data. 

N = 10000
mp = ModelParams(:np, :all, 50, 7, 150)
results_alltests = zeros(Float64, mp.exp_days, N)
results_firsttests = zeros(Float64, mp.exp_days, N)
println("using threads: $(Threads.nthreads())")
Threads.@threads for i = 1:N    
    res = run_scenario(mp, fc)
    results_alltests[:, i] = res.alltests
    results_firsttests[:, i] = res.firsttest
end
println(mp)


using threads: 8
ModelParams
  test_type: Symbol np
  inf_type: Symbol all
  firsttest: Int16 50
  testfreq: Int16 7
  exp_days: Int32 150



In [9]:
resvec = vec(results_alltests[1:78, :])
posidx = findall(x -> x > 0, resvec)
resvec = resvec[posidx]
println("full vector length: $(length(results_alltests)), filtered length: $(length(posidx))")
println("mean of the vector: $(mean(resvec))")

# conver to 500 bins of data. 
minel=Int64(floor(length(posidx)/500));
println("bins: $minel")
rnp = zeros(Float64, 500)
for jj=1:500
    rnp[jj] = mean(resvec[((jj-1)*minel+1):(jj*minel)]);
end
rnp
println("mean of the binn'ed vector: $(mean(rnp))")

mnhist = hist(rnp)
@gp "set term svg enhanced standalone mouse size 800,400" ## use jupyter
@gp :- "set style histogram cluster gap 2"
#@gp :- "set style fill solid 0.1 border -1"
@gp :- "set xlabel 'case detection percentage'"
@gp :- "set ylabel 'counts'"
#@gp :- "set boxwidth 0.9"
#@gp :- histdata.bins histdata.counts "w fillsteps notitle lc rgb 'red'"
@gp :- mnhist.bins mnhist.counts "w boxes smooth freq notitle lc rgb 'black'"


full vector length: 1500000, filtered length: 490505
mean of the vector: 0.9409849295699968
bins: 981
mean of the binn'ed vector: 0.9409844495420333
